In [11]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
pages = [''] + [f'page/{num}' for num in range(2, 12)]
url = "https://comidadibuteco.com.br/butecos/belo-horizonte/"

first_links = []

for page in pages:
    # Fetch the HTML content
    response = requests.get(url + page)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all divs with class 'item'
    items = soup.find_all('div', class_='item')

    # Extract the first link from each 'item' div
    for item in items:
        first_link = item.find('a')
        if first_link and 'href' in first_link.attrs:
            first_links.append(first_link['href'])

In [3]:
# Print the results
for link in first_links:
    print(link)

https://comidadibuteco.com.br/buteco/alexandres-bar/
https://comidadibuteco.com.br/buteco/amarelim-do-prado/
https://comidadibuteco.com.br/buteco/andrades-beer/
https://comidadibuteco.com.br/buteco/arcos-bar/
https://comidadibuteco.com.br/buteco/armazem-santa-amelia/
https://comidadibuteco.com.br/buteco/avalanche-espeteria/
https://comidadibuteco.com.br/buteco/azougue-fogo-e-bar/
https://comidadibuteco.com.br/buteco/baiuca/
https://comidadibuteco.com.br/buteco/3231-revision-v1/
https://comidadibuteco.com.br/buteco/bar-bendita-baderna/
https://comidadibuteco.com.br/buteco/bar-da-cintia/
https://comidadibuteco.com.br/buteco/bar-da-fia/
https://comidadibuteco.com.br/buteco/bar-da-lu/
https://comidadibuteco.com.br/buteco/bar-da-silvania/
https://comidadibuteco.com.br/buteco/bar-do-bartolomeu/
https://comidadibuteco.com.br/buteco/bar-do-bem/
https://comidadibuteco.com.br/buteco/bar-do-kim/
https://comidadibuteco.com.br/buteco/bar-do-nelson/
https://comidadibuteco.com.br/buteco/bar-do-primo/

In [9]:
butecos = {}

for link in first_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the target div
    section = soup.find('div', class_='section-text')
    title = soup.find('h1').get_text(strip=True)

    # Create the dictionary
    data = {}

    first_paragraph = True
    # Loop through each paragraph
    for p in section.find_all('p'):
        bold_tag = p.find('b')
        if bold_tag:
            key = bold_tag.get_text(strip=True).rstrip(':')  # Remove colon if present
            bold_tag.extract()  # Remove the <b> tag from the paragraph to get the value
            value = p.get_text(strip=True)
            if first_paragraph:
                data['Prato'] = key
                data['Descricao'] = value
                first_paragraph = False
            else:
                data[key] = value

    butecos[title] = data

In [10]:
print(butecos)

{'Alexandre’s Bar': {'Prato': 'O fabuloso discão', 'Descricao': 'Polpetone com recheio de muçarela acompanhada com caponata de berinjela e abobrinha.', 'Endereço': 'Rua David Alves do Vale, 68 | Santa Rosa, Belo Horizonte – MG', 'Complemento': 'Loja 01', 'Telefone': '31997753957', 'Horario': 'Terça, das 17h às 23h | Quarta, das 17h às 23h | Quinta, das 17h às 23h | Sexta, das 17h às 23h | Sábado, das 12h às 22h | Domingo, das 12h às 22h |'}, 'Amarelim do Prado': {'Prato': 'Trem de roças', 'Descricao': 'Bochecha de porco na cama de angu mole, pesto de horta, mandioca e agrião. Acompanha pão e patê de fígado de galinha.', 'Endereço': 'Avenida Francisco Sá, 658 | Prado, Belo Horizonte – MG', 'Telefone': '31 981084068 / 31 25273847', 'Horario': 'Segunda, das 11h à 0h | Terça, das 11h à 0h | Quarta, das 11h à 0h | Quinta, das 11h à 0h | Sexta, das 11h à 0h | Sábado, das 11h à 0h | Domingo, das 11h às 22h |'}, 'Andrade’s Beer': {'Prato': 'Chora canelinha', 'Descricao': 'Ossobuco com purê de 

In [15]:
fieldnames = set()
for subdict in butecos.values():
    fieldnames.update(subdict.keys())

# Optional: Add outer keys as an ID column
fieldnames = ['Estabelecimento'] + sorted(fieldnames)

# Step 2: Write to CSV
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for outer_key, inner_dict in butecos.items():
        row = {'Estabelecimento': outer_key}
        row.update(inner_dict)
        writer.writerow(row)

In [17]:
df = pd.read_csv('output.csv')
df

,Estabelecimento,Complemento,Descricao,Endereço,Horario,Prato,Telefone
0,Alexandre’s Bar,Loja 01,Polpetone com recheio de muçarela acompanhada ...,"Rua David Alves do Vale, 68 | Santa Rosa, Belo...","Terça, das 17h às 23h | Quarta, das 17h às 23h...",O fabuloso discão,31997753957
1,Amarelim do Prado,NaN,"Bochecha de porco na cama de angu mole, pesto ...","Avenida Francisco Sá, 658 | Prado, Belo Horizo...","Segunda, das 11h à 0h | Terça, das 11h à 0h | ...",Trem de roças,31 981084068 / 31 25273847
2,Andrade’s Beer,NaN,"Ossobuco com purê de batatas, acompanhado de t...","Rua Dona Geni, 32 | Jardim Florência (Venda No...","Segunda, das 16h às 23h | Terça, das 16h às 23...",Chora canelinha,31989145104
3,Arcos Bar,Loja 5,"Acém cozido com ervas finas, acompanhado com b...","Rua da Bahia, 1144 | Centro, Belo Horizonte – MG","Terça, das 15h às 2h | Quarta, das 15h às 2h |...",Panela da Diversidade,31984411161
4,Armazém Santa Amélia,NaN,Polenta cremosa com ragu de carnes cheio de sa...,"Avenida da Sinfonia, 253 | Santa Amélia, Belo ...","Segunda, das 17h à 1h | Terça, das 17h à 1h | ...",Segredo do Armazém,31972174192
...,...,...,...,...,...,...,...
119,Xambar,"Edifício Maleta, 2º Andar, Lojas 7 e 10","Miolo de acém assado na pressão, acompanhado d...","Avenida Augusto de Lima, 233 | Centro, Belo Ho...","Segunda, das 17h à 0h | Terça, das 17h à 0h | ...",Mineirin Oriental,31994108635
120,Xico da Kafua,NaN,"Mão suína, acompanhada de farofa crocante de s...","Avenida Itaú, 1195 – Loja A | João Pinheiro, B...","Segunda, das 11h30 às 16h30 | Terça, das 11h30...",Pé na areia,3133752640
121,Xico do Churrasco,NaN,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares 920 – Castelo, Belo...",Segunda à sexta de 17h à 0h | Sabado de 12h à 0h,Xico Balaio,31993643195
122,Zé Bolacha,NaN,Bolinho de costelinha desfiada com requeijão e...,"Rua das Melancias, 35 | Vila Cloris, Belo Hori...","Segunda, das 18h à 0h | Terça, das 18h à 0h | ...",Rabicó,3134541052
